In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.draw import draw_squares
from utils.square import SquareDataset
from torch.utils.data import DataLoader

In [2]:
class LinearModel(nn.Module):
    def __init__(self, x, y):
        super(LinearModel, self).__init__()
        self.layer1 = nn.Linear(x, y)

    def forward(self, x):
        x = self.layer1(x)
        return F.softmax(x, dim=1)

In [3]:
squares = SquareDataset(3000)
for i in range(5):
    print(squares[i])

(tensor([202., 165., 149., 158., 162., 241., 102.,  35., 190.]), tensor([0., 1., 0.]))
(tensor([208., 150., 194., 206.,  58., 119.,  42., 196., 168.]), tensor([1., 0., 0.]))
(tensor([ 70.,  13., 102., 147., 249., 243.,  70., 134.,  51.]), tensor([0., 1., 0.]))
(tensor([ 66.,  24.,  19., 161., 155., 231., 251., 196., 198.]), tensor([0., 0., 1.]))
(tensor([  5.,  79., 211.,  75., 102., 122., 215., 173., 104.]), tensor([0., 0., 1.]))


# Setting things up!

In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Use the nn package to define our model and loss function.
model = LinearModel(9, 3)
model = model.to(device)

cost = torch.nn.BCELoss()

# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# dataset!
dataloader = DataLoader(squares, batch_size=128)

epochs = 350

# The Optimization Loop

In [6]:
for t in range(epochs):
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device) / 255, Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

    if t % 50 == 0:
        print('l: {:>8f}, (e {:>3})'.format(loss.item(), t))
        
print('l: {:>8f}, (e {:>3})\nDone!'.format(loss.item(), t))

l: 0.621107, (e   0)
l: 0.355156, (e  50)
l: 0.246422, (e 100)
l: 0.191357, (e 150)
l: 0.158104, (e 200)
l: 0.135935, (e 250)
l: 0.120208, (e 300)
l: 0.108736, (e 349)
Done!


In [7]:
print("W's and b's:")
for p in model.parameters():
    print(p)

W's and b's:
Parameter containing:
tensor([[ 4.1006,  4.2217,  3.7121, -2.6562, -2.9505, -3.1881, -2.4599, -2.7355,
         -2.7374],
        [-2.4797, -2.4096, -2.9708,  3.8162,  3.4323,  3.4614, -2.4201, -2.5883,
         -2.4835],
        [-2.5055, -2.4078, -2.8995, -2.5732, -2.8789, -2.8371,  3.9360,  3.7303,
          3.7653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.0701, 0.1407, 0.1582], device='cuda:0', requires_grad=True)


# Trying it out (inference)

In [8]:
img = [255, 255, 145, 255, 255, 255, 232, 32, 255]
with torch.no_grad():
    o = model(torch.tensor(img, dtype=torch.float).view(1, 9).to(device) / 255)
    
print(o)
print('\ntop:    {:>8f}\nmiddle: {:>8f}\nbottom: {:>8f}'.format(*o[0]))

tensor([[0.0534, 0.9445, 0.0020]], device='cuda:0')

top:    0.053439
middle: 0.944526
bottom: 0.002035
